In [2]:
%autosave 300
%reload_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [3]:
import os

os.chdir(
    r"/mnt/batch/tasks/shared/LS_root/mounts/clusters/copilot-model-run/code/Users/Soutrik.Chowdhury/abi_hackathon_2k24"
)
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/copilot-model-run/code/Users/Soutrik.Chowdhury/abi_hackathon_2k24


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", None)
from pandarallel import pandarallel
import re
from datetime import datetime
import gc
from sklearn.preprocessing import FunctionTransformer

# from helpers.compress import reduce_mem_usage
import pickle
import warnings
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
warnings.filterwarnings("ignore")

In [5]:
data_folder = "data"
macro_data_folder = "macro_data"
processed_data_folder = "processed_data_S2"
processed_macro = "macro_processed"
fe_data = "feature_engineered_data"

In [6]:
volume_monthly_df_best = pd.read_pickle(
    os.path.join(
        os.getcwd(), processed_data_folder, "submission_id_vol_cluster_monthly_df_all_feats.pkl"
    )
)

In [7]:
submission_df = pd.read_csv(
    os.path.join(
        os.getcwd(), data_folder, "sample_submission_file.csv"
    )
)

In [8]:
test_cut_date = "2023-01-01"
full_train_df = volume_monthly_df_best.loc[volume_monthly_df_best['year_month_start_date']<test_cut_date].reset_index(drop=True)
test_df =  volume_monthly_df_best.loc[volume_monthly_df_best['year_month_start_date']>=test_cut_date].reset_index(drop=True)

In [9]:
full_train_df.shape

(1973838, 67)

In [10]:
test_df.shape

(592044, 67)

In [11]:
# test_df['poc_ppg_id'].nunique() *12

In [12]:
val_date = "2022-07-01"
train_df = full_train_df.loc[full_train_df['year_month_start_date']<val_date].reset_index(drop=True)
val_df = full_train_df.loc[full_train_df['year_month_start_date']>=val_date].reset_index(drop=True)

In [13]:
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.common import space

In [14]:
label = 'sales_volume_hl_agg'

In [15]:
y_val = val_df[label]
val_df_nolabel = val_df.drop(columns=[label])

In [16]:
metric='smape'

In [17]:
# GBM,XGB

In [18]:
gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': space.Int(lower=100, upper=1000, default=200), 
    'num_leaves': space.Int(lower=26, upper=66, default=36),
    'learning_rate' : space.Real(0.01, 0.1, log=True)

}
xgb_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'max_depth': space.Int(lower=6, upper=15, default=10),
    'eta' : space.Real(0.01, 0.5, log=True)
}
xt_options = {  # specifies non-default hyperparameter values for neural network models
    'min_samples_split': space.Int(lower=2, upper=8, default=5), 
    'n_estimators': space.Int(lower=100, upper=300, default=100),
}

In [ ]:
hyperparameters = {  # hyperparameters of each model type
                   'GBM': gbm_options,
                   'XGB': xgb_options, 
                    "XT":{}
                  }  
num_trials = 15  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}  # Refer to TabularPredictor.fit docstring for all valid values

predictor = TabularPredictor(label=label, eval_metric=metric).fit(
    full_train_df,
    hyperparameters=hyperparameters,
    hyperparameter_tune_kwargs=hyperparameter_tune_kwargs, 
#     presets="high_quality",
#     refit_full=True

)

No path specified. Models will be saved in: "AutogluonModels/ag-20240719_140049"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #73~20.04.1-Ubuntu SMP Mon May 6 09:43:44 UTC 2024
CPU Count:          16
Memory Avail:       76.04 GB / 108.07 GB (70.4%)
Disk Space Avail:   5035.03 GB / 5120.00 GB (98.3%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality'

  0%|          | 0/15 [00:00<?, ?it/s]

Fitted model: LightGBM/T1 ...
	-0.5578	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	24.02s	 = Training   runtime
	0.04s	 = Validation runtime
Fitted model: LightGBM/T2 ...
	-0.5099	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	45.8s	 = Training   runtime
	0.11s	 = Validation runtime
Fitted model: LightGBM/T3 ...
	-0.5019	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	117.83s	 = Training   runtime
	0.76s	 = Validation runtime
Fitted model: LightGBM/T4 ...
	-0.4983	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	118.62s	 = Training   runtime
	0.68s	 = Validation runtime
Fitted model: LightGBM/T5 ...
	-0.5058	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	104.31s	 = Training   runtime
	0.49s	 = Validation runtime
Fitted model: LightGBM/T6 ...
	-0.51	 = Validation score   (-symmetric_mean_absolute_percentage_error)
	92.08s	 = Training   runtime
	0.47s	 = Validation runtime
Fitted model: Li

  0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
# predictor = TabularPredictor.load('AutogluonModels/ag-20240719_063409')

In [ ]:
y_pred = predictor.predict(val_df_nolabel)
print("Predictions:  ", list(y_pred)[:5])
perf = predictor.evaluate(val_df, auxiliary_metrics=False)

In [ ]:
predictor.leaderboard(val_df)

In [ ]:
results = predictor.fit_summary()

In [ ]:
predictor.evaluate(val_df)

In [ ]:
predictor.feature_importance(test_data)

###### Predictions

In [ ]:
test_df_id = test_df[['ppg_name','unique_poc_id','year','month','quarter']]

In [ ]:
y_pred_test = predictor.predict(test_df)

In [ ]:
test_df_id['quarterly_sales_hl'] = y_pred_test

In [ ]:
test_df_id['quarterly_sales_hl'].mean()

In [ ]:
test_df_id.head()

In [ ]:
test_df_id = test_df_id.groupby(['ppg_name','unique_poc_id','year','quarter','month'],as_index=False).agg({"quarterly_sales_hl":"mean"})

In [ ]:
test_df_id['quarterly_sales_hl'].mean()

In [ ]:
test_df_id_qrtr = test_df_id.groupby(['ppg_name','unique_poc_id','year','quarter'],
                                     as_index=False).agg({"quarterly_sales_hl":"sum"})

In [ ]:
test_df_id_qrtr.shape

In [ ]:
# submission_df = submission_df.drop(['quarterly_sales_hl'],axis=1) 

In [ ]:
submission_df_merge = pd.merge(submission_df,test_df_id_qrtr,
                               on=["ppg_name","unique_poc_id","year","quarter"],
                              how='left')

In [ ]:
submission_df_merge.isnull().sum()

In [ ]:
submission_df_merge['quarterly_sales_hl'].sum()

In [59]:
iter_name = 2
os.makedirs(os.path.join(os.getcwd(),"sample_submissions"))
submission_df_merge.to_csv(os.path.join(os.getcwd(),"sample_submissions",f'Submission_{iter_name}.csv'),index=False)